In [ ]:
import yaml
from pathlib import Path
import h5py
import matplotlib.pyplot as plt

# Get project root
notebook_dir = Path.cwd()
project_root = Path('pytorch/crypto/classifier')

# Load config
with open(project_root / 'config' / 'config.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Generate HDF5 path from config
symbol = config['data']['symbol']
interval = config['data']['interval']
start_date = config['data']['start_date']
end_date = config['data']['end_date']

img_filename = f"crypto_{symbol}_{interval}_{start_date}_{end_date}.h5"
img_path = project_root / config['paths']['processed_data'] / img_filename

# Load and display first image
if img_path.exists():
    with h5py.File(img_path, 'r') as f:
        img = f['images'][0]
        print(f"Total images in file: {f['images'].shape[0]:,}")
        print(f"Image resolution: {img.shape[1]}x{img.shape[0]} (WxH)")
    
    plt.figure(figsize=(12, 7))
    plt.imshow(img, cmap='gray', aspect='auto')
    plt.axis('off')  # Remove all axes and labels
    plt.tight_layout()
    plt.show()
else:
    print(f"❌ HDF5 file not found: {img_path}")
    print("Run: python pipeline.py")